In [ ]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import os
import glob
import json
import ast
from datetime import datetime, timedelta
import random

# Custom Function

In [ ]:
def get_type(product_category):
    if pd.isna(product_category):
        return 'Misc'
    else:
        return product_category.split('>')[-1]

In [ ]:
def convert_weight(weight):
    if pd.isna(weight):
        return ''
    else:
        grams = weight/2205
        return float("{:.2f}".format(grams))

In [ ]:
def get_option_name(input_string):
    if pd.isna(input_string):
        return input_string
    else:
        input_string = input_string.replace('"','')
        sections = input_string.strip().split('\n\n')
        option = sections[0].strip().split('\n')
        option_name = option[0]
        return option_name

In [ ]:
def get_option_value(input_string):
    if pd.isna(input_string):
        return input_string
    else:
        input_string = input_string.replace('"','')
        sections = input_string.strip().split('\n\n')
        option = sections[0].strip().split('\n')
        option_values = option[1:-1]
        return option_values

In [ ]:
def get_variant_sku(variant_sku, opt_value):
    if (pd.isna(variant_sku)) | (pd.isna(opt_value)):
        return variant_sku
    else:
        return variant_sku + '-' + opt_value

In [ ]:
def get_image_alt_text(image_src):
    if pd.isna(image_src):
        return image_src
    else:
        return image_src.split('/')[-1]

In [ ]:
def convert_upc(barcode):
    if (pd.isna(barcode)) | (barcode == 'DOES NOT APPLY'):
        return ''
    else:
        return str(int(barcode))

In [ ]:
def read_all(folder_path, encoding='utf-8'):

    path = os.path.join(os.getcwd(), folder_path)
    all_files = glob.glob(os.path.join(path , "*.csv"))
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, encoding=encoding)
        li.append(df)
    
    frame = pd.concat(li, axis=0, ignore_index=True)

    return frame

In [ ]:
def get_tag(breadcrumb):
    if pd.isna(breadcrumb):
        return breadcrumb
    else:
        tags = breadcrumb.replace(',', ';').split(':')
        result = ','.join(tags)
        return result

In [ ]:
def get_collection(tags):
    if pd.isna(tags):
        return tags
    else:
        collection = tags.split(',')
        return collection[-1]

In [ ]:
def gen_datetime(x, min_year=2020, max_year=datetime.now().year):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=((365 * years) - 166)) 
    result_datetime = start + (end - start) * random.random()
    return result_datetime.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
def extract_data(dict_string, field):
    dict_data = ast.literal_eval(dict_string)
    results = list()
    for data in dict_data['data']:
        results.append(data[field])
    return results

In [ ]:
def generate_tt_product_url(handle):
    if pd.isna(handle):
        return None
    return f'https://www.trendtimes.com/{handle}.html'

In [ ]:
def generate_sf_product_url(handle):
    if pd.isna(handle):
        return None
    return f'https://www.trendtimes.com/products/{handle}'

In [ ]:
def generate_sf_collection_url(handle):
    if pd.isna(handle):
        return None
    return f'https://www.trendtimes.com/collections/{handle}'

In [ ]:
def generate_key(product_id):
    if pd.isna(product_id):
        return None
    product_id = product_id.strip('-')
    product_id = re.sub(r'-+', '-', product_id)
    if product_id.startswith('-'):
        product_id = product_id[1:]
    
    return product_id

In [ ]:
def format_price(price):
    if pd.isna(price):
        return None
    if type(price) == float:
        return price
    price = price.split(' ')[0]
    
    return price

In [ ]:
def format_weight(weight):
    if pd.isna(price):
        return None
    if (type(price) == float) & (type(price) == int):
        return price
    price = float(price.replace(' ', '.'))
    
    return price

In [ ]:
def generate_collection_url(handle):
    if pd.isna(handle):
        return None
    return f'https://www.trendtimes.com/{handle}.html'    

In [ ]:
def extract_alphanumeric(text):
    pattern = re.compile(r"\b[a-zA-Z0-9]+\b|\'")
    alphanumeric_matches = pattern.findall(text)
    return alphanumeric_matches

In [ ]:
def title_to_id(title):
    if pd.isna(title):
        return None
    result = extract_alphanumeric(title.lower())
    result = '-'.join(result)
    result = re.sub(r'-+', '-', result)
    result = result.replace("-'-", "")
    return result

parsed_dict = parse_string_to_dict(input_string)
print(parsed_dict)

# Create shopify template

In [ ]:
template = pd.read_csv('products_export.csv')

In [ ]:
template.columns

# Transform source product datas into shopify template

In [ ]:
source = read_all('source', encoding='latin-1')

In [ ]:
trendtime_col = source[source['supplier'] == 'Sections']

In [ ]:
trendtime_prod = source[source['supplier'] != 'Sections']

# unimported_items

In [ ]:
ui_df = pd.read_csv('unimported_items.csv')

In [ ]:
ui_df['id'] = ui_df['product_urls'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [ ]:
ui_id_list = ui_df['id'].to_list() 

In [ ]:
source = source[source['id'].isin(ui_id_list)].reset_index(drop=True)

In [ ]:
result = pd.DataFrame(columns=template.columns)

In [ ]:
result['Handle'] = source.loc[:,'id']

In [ ]:
result.loc[:, 'Title'] = source['name']
result.loc[:, 'Title'] = result.loc[:, 'Title'].fillna('No Title')

In [ ]:
result.loc[:, 'Body (HTML)'] = source['caption']

In [ ]:
result.loc[:, 'Vendor'] = source['brand']

In [ ]:
result.loc[:, 'Product Category'] = source['google-base-product-type']

In [ ]:
result.loc[:, 'Type'] = source['google-base-product-type'].apply(get_type)

In [ ]:
# bc_df = read_all('breadcrumbs')
bc_df = pd.read_csv('breadcrumbs/breadcrumbds_data_unimported.csv')

In [ ]:
bc_df['id'] = bc_df['id'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [ ]:
bc_df.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
result.loc[:, 'Tags'] = pd.merge(result.loc[:, 'Handle'], bc_df, how='left', left_on='Handle', right_on='id')['breadcrumbs']
result.loc[:, 'Tags'] = result.loc[:, 'Tags'].apply(get_tag)

In [ ]:
result.loc[:, 'Published'] = True

In [ ]:
result.loc[:, 'Option1 Name'] = source['options'].apply(get_option_name)

In [ ]:
result.loc[:, 'Option1 Value'] = source['options'].apply(get_option_value)

In [ ]:
result.loc[:, 'Variant SKU'] = source['code']

In [ ]:
result.loc[:, 'Variant Grams'] = source['ship-weight']

In [ ]:
result.loc[:, 'Variant Inventory Tracker'] = 'shopify'

In [ ]:
result.loc[:, 'Variant Inventory Qty'] = source['orderable'].replace({'Yes': 10, 'No': 0})

In [ ]:
result.loc[:, 'Variant Inventory Policy'] = 'deny'

In [ ]:
result.loc[:, 'Variant Fulfillment Service'] = 'manual'

In [ ]:
result.loc[:, 'Variant Price'] = source['sale-price'].apply(format_price)

In [ ]:
result.loc[:, 'Variant Compare At Price'] = source['price'].apply(format_price)

In [ ]:
result.loc[:, 'Variant Requires Shipping'] = source['need-ship'].replace({'Yes': True, 'No': False})

In [ ]:
result.loc[:, 'Variant Taxable'] = True

In [ ]:
result.loc[:, 'Variant Barcode'] = source['upc']

In [ ]:
result.loc[:, 'Variant Barcode'] = result.loc[:, 'Variant Barcode'].apply(convert_upc)

In [ ]:
image_df = pd.read_csv('images_src_rev2/images_data_unimported.csv')

In [ ]:
image_df['id'] = image_df['id'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [ ]:
image_df[image_df.duplicated()]

In [ ]:
image_df.drop_duplicates(inplace=True)

In [ ]:
result.loc[:, 'Image Src'] = pd.merge(result.loc[:, 'Handle'], image_df, how='left', left_on='Handle', right_on='id')['images']

In [ ]:
result.loc[:, 'Gift Card'] = source['gift-certificate'].replace({'Yes': True, 'No': False})

In [ ]:
result.loc[:, 'SEO Title'] = source['name']

In [ ]:
result.loc[:, 'SEO Description'] = source['abstract']

In [ ]:
result.loc[:, 'Google Shopping / Google Product Category'] = source['google-base-product-type']

In [ ]:
result.loc[:, 'Google Shopping / MPN'] = source['upc']

In [ ]:
result.loc[:,'Google Shopping / MPN'] = result.loc[:,'Google Shopping / MPN'].apply(convert_upc)

In [ ]:
result.loc[:, 'Google Shopping / Condition'] = 'New'

In [ ]:
result.loc[:, 'Variant Weight Unit'] = 'lb'

In [ ]:
result.loc[:, 'Included / United States'] = True

In [ ]:
result.loc[:, 'Included / Canada'] = True

In [ ]:
result.loc[:, 'Status'] = 'active'

In [ ]:
# result.loc[:,'Collection'] = result.loc[:,'Tags'].apply(get_collection)

# Input After Explode Option

In [ ]:
exploded_opt_result = result.assign(Option1Value=result['Option1 Value'].str.split(";")).explode("Option1 Value")

In [ ]:
exploded_opt_result[~pd.isna(exploded_opt_result['Option1 Name'])][['Handle', 'Option1 Name', 'Option1 Value', 'Variant SKU', 'Image Src']]

In [ ]:
exploded_opt_result.loc[(~pd.isna(exploded_opt_result['Option1 Name'])) & (pd.isna(exploded_opt_result['Option1 Value'])), 'Option1 Value'] = 'Standard'

In [ ]:
exploded_opt_result.loc[:, 'Variant SKU'] = exploded_opt_result.apply(lambda x: get_variant_sku(x['Variant SKU'], x['Option1 Value']), axis=1)

In [ ]:
uneeded_opt_column = [
    'Title', 'Body (HTML)', 'Vendor', 'Product Category', 'Type', 'Tags',
    'Published', 'Option1 Name', 'Option2 Name',
    'Option2 Value', 'Option3 Name', 'Option3 Value', 'SEO Title','SEO Description', 'Image Src',
    'Google Shopping / Google Product Category', 'Google Shopping / Gender',
    'Google Shopping / Age Group', 'Google Shopping / MPN',
    'Google Shopping / Condition', 'Google Shopping / Custom Product',
    'Google Shopping / Custom Label 0', 'Google Shopping / Custom Label 1',
    'Google Shopping / Custom Label 2', 'Google Shopping / Custom Label 3',
    'Google Shopping / Custom Label 4', 'Cost per item', 'Included / United States', 'Price / United States', 
    'Compare At Price / United States', 'Included / Canada', 'Price / Canada',
    'Compare At Price / Canada', 'Status'
]

In [ ]:
exploded_opt_result['Flag'] = exploded_opt_result['Handle'].eq(exploded_opt_result['Handle'].shift(1))

In [ ]:
exploded_opt_result.loc[exploded_opt_result['Flag'], uneeded_opt_column] = ''

In [ ]:
exploded_opt_result.drop('Flag', axis=1, inplace=True)

# Input After Explode Images Src

In [ ]:
final_df = exploded_opt_result.copy()

In [ ]:
final_df['Image Src'] = final_df['Image Src'].str.split(';')

In [ ]:
final_df = final_df.assign(ImageSrc=final_df['Image Src']).explode("Image Src")

In [ ]:
final_df.drop(['ImageSrc', 'Option1Value'], inplace=True, axis=1)

In [ ]:
final_df['Image Alt Text'] = final_df['Image Src'].apply(get_image_alt_text)

In [ ]:
final_df.fillna('', inplace=True)

In [ ]:
uneeded_image_column = [
    'Title', 'Body (HTML)', 'Vendor', 'Product Category', 'Type',
    'Tags', 'Published', 'Option1 Name', 'Option1 Value', 'Option2 Name',
    'Option2 Value', 'Option3 Name', 'Option3 Value', 'Variant SKU',
    'Variant Grams', 'Variant Inventory Tracker', 'Variant Inventory Qty',
    'Variant Inventory Policy', 'Variant Fulfillment Service',
    'Variant Price', 'Variant Compare At Price',
    'Variant Requires Shipping', 'Variant Taxable', 'Variant Barcode',
    'Image Position', 'Gift Card',
    'SEO Title', 'SEO Description',
    'Google Shopping / Google Product Category', 'Google Shopping / Gender',
    'Google Shopping / Age Group', 'Google Shopping / MPN',
    'Google Shopping / Condition', 'Google Shopping / Custom Product',
    'Google Shopping / Custom Label 0', 'Google Shopping / Custom Label 1',
    'Google Shopping / Custom Label 2', 'Google Shopping / Custom Label 3',
    'Google Shopping / Custom Label 4', 'Variant Image',
    'Variant Weight Unit', 'Variant Tax Code', 'Cost per item',
    'Included / United States', 'Price / United States',
    'Compare At Price / United States', 'Included / Canada',
    'Price / Canada', 'Compare At Price / Canada', 'Status'
]

In [ ]:
final_df['Flag'] = (final_df['Handle'].eq(final_df['Handle'].shift(1))) & (final_df['Option1 Value'].eq(final_df['Option1 Value'].shift(1)))

In [ ]:
final_df.loc[final_df['Flag'], uneeded_image_column] = ''

In [ ]:
final_df.drop('Flag', axis=1, inplace=True)

In [ ]:
final_df

# Explode Collection

In [ ]:
very_final_df = final_df.copy()

In [ ]:
very_final_df['Tags_2'] = very_final_df['Tags'].str.split(',')

In [ ]:
very_final_df = very_final_df.assign(Collection=very_final_df['Tags_2']).explode("Collection")

In [ ]:
uneeded_image_column = [
    'Title', 'Body (HTML)', 'Vendor', 'Product Category', 'Type',
    'Tags', 'Published', 'Option1 Name', 'Option1 Value', 'Option2 Name',
    'Option2 Value', 'Option3 Name', 'Option3 Value', 'Variant SKU',
    'Variant Grams', 'Variant Inventory Tracker', 'Variant Inventory Qty',
    'Variant Inventory Policy', 'Variant Fulfillment Service',
    'Variant Price', 'Variant Compare At Price', 'Image Src',
    'Variant Requires Shipping', 'Variant Taxable', 'Variant Barcode',
    'Image Position', 'Gift Card', 'Image Alt Text',
    'SEO Title', 'SEO Description',
    'Google Shopping / Google Product Category', 'Google Shopping / Gender',
    'Google Shopping / Age Group', 'Google Shopping / MPN',
    'Google Shopping / Condition', 'Google Shopping / Custom Product',
    'Google Shopping / Custom Label 0', 'Google Shopping / Custom Label 1',
    'Google Shopping / Custom Label 2', 'Google Shopping / Custom Label 3',
    'Google Shopping / Custom Label 4', 'Variant Image',
    'Variant Weight Unit', 'Variant Tax Code', 'Cost per item',
    'Included / United States', 'Price / United States',
    'Compare At Price / United States', 'Included / Canada',
    'Price / Canada', 'Compare At Price / Canada', 'Status'
]

In [ ]:
very_final_df.drop('Tags_2', axis=1, inplace=True)

In [ ]:
very_final_df.fillna('', inplace=True)

In [ ]:
very_final_df['Flag'] = (very_final_df['Handle'].eq(very_final_df['Handle'].shift(1))) & (very_final_df['Collection'] != '')

In [ ]:
very_final_df.loc[very_final_df['Flag'], uneeded_image_column] = ''

In [ ]:
very_final_df.drop('Flag', axis=1, inplace=True)

In [ ]:
very_final_df.to_csv('result/trendtime_shopify_products_unimported.csv', index=False)

# Data to CSV

In [ ]:
collection_df.to_csv('Collection_list_unimported.csv')

In [ ]:
# final_df.iloc[10002:15001].to_csv('result/trendtime_shopify_products_03.csv', index=False)
final_df.to_csv('result/trendtime_shopify_products_unimported.csv', index=False)

In [ ]:
final_df[final_df['Handle'] == 'aa-batteries-4-pack']

In [ ]:
default_df = pd.read_csv('default-table (copy 1).csv', encoding='latin-1')

In [ ]:
default_df.loc[default_df['id'].str.contains("huge-apache-rc-helicopter")]

In [ ]:
item_2

# Review converter

In [ ]:
review_df = read_all('reviews')

In [ ]:
review_df = review_df[~pd.isna(review_df['reviews'])]

In [ ]:
review_df.reset_index(inplace=True, drop=True)

In [ ]:
review_df

In [ ]:
review_template = pd.read_csv('Okendo_Import_Template_v1(1).csv')

In [ ]:
review_template

In [ ]:
review_result = review_template[0:0]

In [ ]:
review_result['handle'] = review_df['id']

In [ ]:
review_result

In [ ]:
review_result.loc[:,'reviews'] = review_df['reviews']

In [ ]:
review_result

In [ ]:
review_result['name'] = review_result.apply(lambda x: extract_data(x['reviews'],field='posted_by'), axis=1)

In [ ]:
review_result['body'] = review_result.apply(lambda x: extract_data(x['reviews'],field='review_text'), axis=1)

In [ ]:
review_result['rating'] = review_result.apply(lambda x: extract_data(x['reviews'],field='review_rating'), axis=1)

In [ ]:
review_result['title'] = review_result.apply(lambda x: extract_data(x['reviews'],field='review_title'), axis=1)

In [ ]:
review_result.loc[0,'reviews']

In [ ]:
review_result

In [ ]:
review_result.drop(columns='reviews', inplace=True)

In [ ]:
review_result = review_result.explode(['name', 'body', 'rating', 'title'], ignore_index=True)

In [ ]:
review_result['dateCreated'] = review_result.apply(lambda x: gen_datetime(x['handle']), axis=1)

In [ ]:
review_result['isApproved'] = True

In [ ]:
review_result

In [ ]:
review_result.to_csv('reviews.csv', index=False)

# Redirct Url

## Product

In [ ]:
trendtime_prod[pd.isna(trendtime_prod['code'])]

In [ ]:
# shopify_redirect_df = pd.read_csv('product_urls_rev1.csv')

In [ ]:
# shopify_redirect_df[shopify_redirect_df['handle'] == 'canadair-rc-plane']

In [ ]:
trendtime_prod['Old Page Url'] = trendtime_prod['id'].apply(generate_tt_product_url)

In [ ]:
trendtime_prod = trendtime_prod[['id', 'Old Page Url']]

In [ ]:
trendtime_prod['key'] = trendtime_prod['id'].apply(generate_key)

In [ ]:
trendtime_prod['New Page Url'] = trendtime_prod['key'].apply(generate_sf_product_url)

In [ ]:
trendtime_prod[['Old Page Url', 'New Page Url']].to_csv('trendtimes-redirectrules-prod.csv', index=False)

In [ ]:
# redirect_urls = source.merge(shopify_redirect_df, how='left', left_on='key', right_on='handle')[['product_urls', 'onlineStorePreviewUrl']]

In [ ]:
# redirect_urls.rename({'product_urls': 'Old Page Url', 'onlineStorePreviewUrl': 'New Page Url'}, inplace=True)

In [ ]:
# redirect_urls.dropna(inplace=True)

In [ ]:
# redirect_urls.to_csv('trendtimes-redirectrules-with-na.csv', index=False)

## Collections

In [ ]:
trendtime_col

In [ ]:
trendtime_col['Old Page Url'] = trendtime_col['id'].apply(generate_tt_product_url)

In [ ]:
trendtime_col = trendtime_col[['id', 'Old Page Url']]

In [ ]:
trendtime_col

In [ ]:
trendtime_col['key'] = trendtime_col['id'].apply(generate_key)

In [ ]:
trendtime_col

In [ ]:
trendtime_col['New Page Url'] = trendtime_col['key'].apply(generate_sf_collection_url)

In [ ]:
sf_collections = pd.read_csv('collection_list.csv') 

In [ ]:
col_checker = trendtime_col.merge(sf_collections, how='left', left_on='key', right_on='handle')

In [ ]:
col_checker['is_available'] = col_checker.apply(lambda x: 'No' if pd.isna(x['handle']) else 'Yes', axis=1)

In [ ]:
trendtime_col

In [ ]:
col_checker.to_csv('trendtimes-redirectrules-col-checker.csv', index=False)

In [ ]:
trendtime_col[['Old Page Url', 'New Page Url']].to_csv('trendtimes-redirectrules-col.csv', index=False)

## Collection Rev1

In [ ]:
tt_col = read_all('sections')

In [ ]:
tt_col

In [ ]:
tt_col = tt_col.drop_duplicates(ignore_index=True)

In [ ]:
tt_col

In [ ]:
tt_col.rename({'url': 'Old Page Url'}, axis=1, inplace=True)

In [ ]:
tt_col['handle'] = tt_col['tag'].apply(title_to_id)

In [ ]:
tt_col['id'] = tt_col['Old Page Url'].apply(lambda x: x.split('/')[-1].replace('.html', ''))

In [ ]:
tt_col

In [ ]:
sf_col = pd.read_csv('sf_collection_list.csv')

In [ ]:
sf_col

### Find Section not appear on breadcrumbs

In [ ]:
section_trendtime_tt = pd.merge(trendtime_col, tt_col, how='left', on='id')

In [ ]:
section_trendtime_tt

In [ ]:
section_trendtime_tt[pd.isna(section_trendtime_tt['handle'])]

In [ ]:
section_tt_trendtime = pd.merge(tt_col, trendtime_col, how='left', on='id')

In [ ]:
section_tt_trendtime

In [ ]:
section_tt_trendtime[pd.isna(section_tt_trendtime['Old Page Url_y'])].to_csv('cek_col_tt_trendtime.csv')

In [ ]:
trendtime_col[trendtime_col['id'].str.contains('halloween-costumes---movie-theater-prop-toys-for-children-to-adults-accessories---makeup-makeup-airb')]

In [ ]:
source[source['id'] == 'halloween-costumes---movie-theater-prop-toys-for-children-to-adults-accessories---makeup-makeup-airb'].to_csv('hallowen.csv', index=False)

In [ ]:
sf_col['New Page Url'] = sf_col['handle'].apply(generate_sf_collection_url)

In [ ]:
sf_col[sf_col['handle'].str.contains('jags')]

In [ ]:
sf_col[sf_col['handle'].str.contains('redcat-volcano-epx-remote-control-truck-parts-epx-pro')]

In [ ]:
tt_col[tt_col['handle'].str.contains('2-channel')]

In [ ]:
redirectrules = pd.merge(tt_col, sf_col, how='left', on='handle')

In [ ]:
redirectrules[pd.isna(redirectrules['New Page Url'])]

In [ ]:
redirectrules = redirectrules[['Old Page Url', 'New Page Url']]

In [ ]:
redirectrules.to_csv('trendtimes-redirectrules-col-rev1.csv', index=False)

## Col Checking

In [ ]:
trendtime_col

In [ ]:
cek_df = pd.merge(trendtime_col, tt_col, how='left', left_on='key', right_on='id')

In [ ]:
cek_df

In [ ]:
cek_df[~pd.isna(cek_df['Old Page Url'])]

In [ ]:
cek_df[cek_df['id_x'].str.contains('2-channel-rc')]

## Col Checking sf

In [ ]:
cek_df1 = pd.merge(tt_col, trendtime_col , how='left', left_on='id', right_on='id')

In [ ]:
cek_df1[pd.isna(cek_df1['key'])]

In [ ]:
trendtime_col[trendtime_col['id'].str.contains('jags')]

In [ ]:
cek_df1[pd.isna(cek_df1['key'])]

# Trial

In [ ]:
source = read_all('source', encoding='latin-1')

In [ ]:
source.shape

In [ ]:
source[source['id'].str.contains('airsoft')].shape

## Collection Trial

In [ ]:
trendtime_col = source[((pd.isna(source['code'])) & (source['supplier'] == 'Sections'))]

In [ ]:
trendtime_col.shape

In [ ]:
trendtime_col['Old Page Url'] = trendtime_col['id'].apply(generate_tt_product_url)

In [ ]:
trendtime_col = trendtime_col[['id', 'Old Page Url']]

In [ ]:
trendtime_col

In [ ]:
scraped_url = read_all('sections')

In [ ]:
scraped_url = scraped_url.drop_duplicates(ignore_index=True)

In [ ]:
scraped_url['id'] = scraped_url['url'].apply(lambda x: x.split('/')[-1].replace('.html', ''))

In [ ]:
scraped_url['handle'] = scraped_url['tag'].apply(title_to_id)

In [ ]:
scraped_url

In [ ]:
trendtime_col['New Page Url'] = trendtime_col['key'].apply(generate_sf_collection_url)

In [ ]:
tt_x_scraped = pd.merge(trendtime_col, scraped_url, how='left', on='id')

In [ ]:
col_checker = trendtime_col.merge(sf_collections, how='left', left_on='key', right_on='handle')

In [ ]:
col_checker['is_available'] = col_checker.apply(lambda x: 'No' if pd.isna(x['handle']) else 'Yes', axis=1)

In [ ]:
tt_x_scraped[~pd.isna(tt_x_scraped['handle'])]

In [ ]:
sf_col = pd.read_csv('sf_collection_list.csv')

In [ ]:
sf_col['New Page Url'] = sf_col['handle'].apply(generate_sf_collection_url)

In [ ]:
sf_col

In [ ]:
result_col = pd.merge(tt_x_scraped, sf_col, how='left', on='handle').reset_index(drop=True)

In [ ]:
result_col

In [ ]:
col_matched = result_col[~pd.isna(result_col['New Page Url'])]

In [ ]:
col_matched[['Old Page Url', 'New Page Url']].to_csv('trendtimes-redirectrules-col-rev2(matched).csv', index=False)

In [ ]:
col_unmatched = result_col[pd.isna(result_col['New Page Url'])]

In [ ]:
col_unmatched[['Old Page Url', 'New Page Url']].to_csv('trendtimes-redirectrules-col-rev2(unmatched).csv', index=False)

## Product Trial

In [ ]:
trendtime_prod = source[~((pd.isna(source['code'])) & (source['supplier'] == 'Sections'))]

In [ ]:
trendtime_prod

In [ ]:
trendtime_prod['Old Page Url'] = trendtime_prod['id'].apply(generate_tt_product_url)

In [ ]:
trendtime_prod['handle'] = trendtime_prod['id'].apply(generate_key)

In [ ]:
trendtime_prod

In [ ]:
trendtime_prod = trendtime_prod[['id', 'handle', 'Old Page Url']]

In [ ]:
trendtime_prod

In [ ]:
sf_prod = pd.read_csv('product_urls_rev2.csv')

In [ ]:
sf_prod[sf_prod['handle'].str.contains('monster-truck-m')]

In [ ]:
sf_prod['New Page Url'] = sf_prod['handle'].apply(generate_sf_product_url)

In [ ]:
sf_prod[['handle','New Page Url']]

In [ ]:
result_prod = pd.merge(trendtime_prod, sf_prod, how='left', on='handle')

In [ ]:
result_prod = result_prod[['Old Page Url', 'New Page Url']]

In [ ]:
prod_matched = result_prod[~pd.isna(result_prod['New Page Url'])]

In [ ]:
prod_matched[['Old Page Url', 'New Page Url']].to_csv('trendtimes-redirectrules-prod-rev2(matched).csv', index=False)

In [ ]:
prod_unmatched = result_prod[pd.isna(result_prod['New Page Url'])]

In [ ]:
prod_unmatched[['Old Page Url', 'New Page Url']].to_csv('trendtimes-redirectrules-prod-rev2(unmatched).csv', index=False)

# Clean Shopify Product

In [ ]:
df = read_all('product_export')

In [ ]:
df

In [ ]:
df_price_0 = df.loc[(pd.isna(df['Image Src'])) & (df['Variant Price'] == 0.0) & (df['Status'] == 'active')]

In [ ]:
df_price_0[df['Handle'] == ]

# Analytics

In [ ]:
df = pd.read_csv('source/item-2.csv', encoding='latin-1')

In [ ]:
df.fillna('', inplace=True)

In [ ]:
df[df['code'] == 'DG88672M'].to_csv('hello_kitty_witch.csv', index=False)

# Cek breadcrumbs order

In [ ]:
breadcrumbs = pd.read_csv('breadcrumbs/breadcrumbs_data.csv')

In [ ]:
breadcrumbs

In [ ]:
breadcrumbs[breadcrumbs['id'] == 'disney-frozen-anna-coronation-4-6'].iloc[0,1]

# Cek Product Dummy without redirect url

In [ ]:
df = read_all('data/product_data')

In [ ]:
df.info()

In [ ]:
df_dummy_product = df.loc[(pd.notna(df['Title'])) & (df['Variant Price'] == 0) & (pd.isna(df['Variant Barcode'])) & (df['Status'] == 'active') & (pd.isna(df['Image Src'])) & (df['Variant Grams'] == 0)]

In [ ]:
df_dummy_product

In [ ]:
df_dummy_product.to_csv('data/dummy_product.csv', index=False)

In [ ]:
collection_df = pd.read_csv('data/collections.csv')

In [ ]:
collection_df

In [ ]:
check_df = df_dummy_product.merge(collection_df, how='left', left_on='Handle', right_on='handle')

In [ ]:
check_df

In [ ]:
check_df.to_csv('data/check.csv', index=False)